Titanic - Machine Learning from Disaster - Kaggle Competition

Importing the libraries that we will be making use of:

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Since we are using Jupyter Notebook and we would like to see plots displayed directly below the code cells that produce them, we must use the following:

In [4]:
%matplotlib inline